In [9]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [36]:
edge_length = 15
cam_params = ((2.0787438565755934, 4.110669876049863, -1.9445352012461516),
 (-0.3012591112717844, -0.2789859717380453, -0.9118167444443614),
 (0.0, 0.0, 0.0))

In [11]:
def get_radius(length, num_sides):
    theta = np.pi * 2 / num_sides
    beta = (np.pi - theta) / 2
    radius = length * np.sin(beta) / np.sin(theta)
    return radius

In [12]:
def generate_polygon_module(poly_size, flat_start = True):
    polygon = []
    radius = get_radius(edge_length, poly_size)
    for i in range(poly_size):
        theta = 2 * np.pi / poly_size * i
        point = np.array([radius * np.sin(theta), radius * np.cos(theta), 0])
        polygon.append(point)

    tips = []
    if flat_start:
        for i in range(poly_size):
            mid = (polygon[i-1] + polygon[i]) / 2
            height = la.norm(polygon[i-1] - polygon[i]) * np.sqrt(3) / 2
            tips.append(mid + mid / la.norm(mid) * height)
    else:
        for i in range(poly_size):
            mid = (polygon[i-1] + polygon[i]) / 2
            height = np.array([0, 0, la.norm(polygon[i-1] - polygon[i]) * np.sqrt(3) / 2])
            tips.append(mid + height)
    
    point_list = polygon + tips
    edge_list = [(i + 1, i + 2) for i in range(poly_size - 1)] + [(poly_size, 1)] + [(i + 1, poly_size + i + 2) for i in range(poly_size - 1)]  + [(poly_size, poly_size + 1)] + [(i + 2, poly_size + i + 2) for i in range(poly_size - 1)] + [(1, poly_size + 1)]
    polygon_path = 'polygon'
    if not os.path.exists(polygon_path):
        os.makedirs(polygon_path)

    with open ('{}/{}_gon.obj'.format(polygon_path, poly_size), 'w') as f:
        for point in point_list:
            f.write('v {} {} {}\n'.format(point[0], point[1], point[2]))
        for edge in edge_list:
            f.write('l {} {}\n'.format(edge[0], edge[1]))

In [13]:
for i in range(11)[3:]:
    generate_polygon_module(i, flat_start = False)

In [44]:
unit_patch.numRestKappaVars()

285

In [46]:
curvature_vector = np.zeros(unit_patch.numRestKappaVars())

In [64]:
def render_polygon(poly_size = 5, curvature = 1):
    cross_section = [3, 0.35]
    model_path = 'polygon/{}_gon.obj'.format(poly_size)
    subdivision_res = 20
    poly_linkage = elastic_rods.RodLinkage(model_path, subdivision_res, False, rod_interleaving_type = InterleavingType.triaxialWeave)
    driver=poly_linkage.centralJoint()
    poly_linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    jdo = poly_linkage.dofOffsetForJoint(0)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint    
#     Set Curvature
    num_curvature_per_edge = int(unit_patch.numRestKappaVars() / (poly_size * 3))
    num_center_curvature = num_curvature_per_edge * poly_size
    phi = curvature / num_curvature_per_edge
    kappa = 2 * np.tan(phi / 2)
    curvature_vector[:num_center_curvature] = np.ones(num_center_curvature) * kappa
    poly_linkage.setRestKappaVars(curvature_vector)
    
    opts = elastic_rods.NewtonOptimizerOptions()
    opts.niter = 10000
    with so():
        elastic_rods.compute_equilibrium(poly_linkage, fixedVars=fixedVars, options = opts)
    new_poly_mod_view = get_double_side_view(poly_linkage, flip = True)
    new_poly_mod_view.setCameraParams(cam_params)
    return poly_linkage, new_poly_mod_view

In [81]:
unit_patch, unit_view = render_polygon(poly_size = 5, curvature = 0.5)

In [82]:
unit_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…